# Simulation of Network Communication Dynamics Model

## Extraction of testing methods

In [10]:
from matplotlib import pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

shapes = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', '+']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'darkgreen', 'chocolate']

In [11]:
RCIs = dict()
DCs = dict()
BCs = dict()
CCs = dict()
PGs = dict()
ERs = dict()
AFOs = dict()
AFPs = dict()

import os

from experiment.functions import func_get_lab_graph
from experiment.lab2._config_ import SoftwareConfit, LabSpace

Graphs = dict()
Labels = dict()
Types = dict()


for softwareName, (status, language, src_dir) in SoftwareConfit.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')

        if not os.path.isdir(labDir):
            os.makedirs(labDir, exist_ok=True)

        G, labelDict, _, typeDict = func_get_lab_graph(modelDir, labDir, softwareName)
        Graphs[softwareName] = G
        Labels[softwareName] = labelDict
        Types[softwareName] = typeDict
        
        RCIs[softwareName] = dict(G.nodes.data('RCI'))
        DCs[softwareName] = dict(G.nodes.data('dc'))
        BCs[softwareName] = dict(G.nodes.data('bc'))
        CCs[softwareName] = dict(G.nodes.data('cc'))
        PGs[softwareName] = dict(G.nodes.data('pg'))
        ERs[softwareName] = dict(G.nodes.data('er'))
        AFOs[softwareName] = dict(G.nodes.data('AFO'))
        AFPs[softwareName] = dict(G.nodes.data('AFP'))

print('Extract data for risk diffusion experiments')

# Diffusion range experiment

## Set the method to be tested and the threshold for testing nodes

In [25]:
# Threshold for IC model testing nodes
ICSimThreshold = 0.05

# The rounds of IC model simulation
ICSimCount = 10

ICThresholds = dict()
ICInfluences = dict()

# Set the excitation threshold and influence for each node
for softwareName, (status, language, src_dir) in SoftwareConfit.items():
    if status:
        G = Graphs[softwareName] 
        ICInfluences[softwareName] = None
        ICThresholds[softwareName] = {node: 0.5 for node in G.nodes}
        pass

# Test method to be tested
TestMethods = [
    ('DRI', RCIs),
    ('DC', DCs),
    ('BC', BCs),
    ('CC', CCs),
    ('PageRank', PGs),
    ('ElementRank', ERs),
]

ICSimDir = os.path.join(LabSpace, '.sim.IC')
if not os.path.isdir(ICSimDir):
    os.mkdir(ICSimDir)
    
  

### Perform simulation

In [ ]:
from experiment.algorithms.propagation_models.independent_cascade import independent_cascade
import pandas as pd
import copy
from pandas import DataFrame
from experiment.functions import func_SortDictByValue


df = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
               [[methodName for methodName, methods in TestMethods], ['max', 'min', 'mean', 'sum']]
           )
)

simResults = dict()
loc =0
for softwareName, (status, language, src_dir) in SoftwareConfit.items():
    if status:
        G = Graphs[softwareName] 
        labels = Labels[softwareName]
        types = Types[softwareName]
        N = G.number_of_nodes()
        
        if isinstance(ICSimThreshold, float) and 0 <= ICSimThreshold <= 1:
            seedsCount = int(G.number_of_nodes() * ICSimThreshold)
        elif isinstance(ICSimThreshold, int) and 0 < ICSimThreshold < N:
            seedsCount = ICSimThreshold
        else:
            seedsCount = int(N * 0.15)
        
        print(f'{softwareName} starts simulating')
        simResults[softwareName] = dict()
        
        for methodName, methods in TestMethods:
            print(f'Simulate method {methodName}')
            method = methods[softwareName]
            SortedMethod = func_SortDictByValue(method)
            SortedMethodIds = list(SortedMethod.keys())
            seeds = SortedMethodIds[:seedsCount]
            seedLabs = [labels[n] for n in seeds]
            seedTypes = [types[n] for n in seeds]
            print(f'\tStart simulating the first {seedsCount} nodes of {methodName}')
            
            simColumns = [f'round {idx}' for idx in range(ICSimCount)]
            itemColumns = ['key', 'val', 'label', 'type']
        
            roundDF = DataFrame(
               columns = pd.MultiIndex.from_product(
                   [simColumns, itemColumns]
               )
            )
            
            sumVal = 0
            maxVal = 0
            minVal = 999999
            meanVal = 0
            
            for idx in range(ICSimCount):
                influences = Influences[softwareName]
                maxItem = 0
            
                simVals = []
                
                for seed in seeds:
                    acNodes, acNodesCounts, newAcNodes = independent_cascade(G, seeds=[seed], influences=influences)
                    simVals.append(acNodesCounts[-1])
                    pass
                
                tSeeds = copy.deepcopy(seeds)
                tSeedLabs = copy.deepcopy(seedLabs)
                tSeedTypes = copy.deepcopy(seedTypes)
                tSeeds.append(-1)
                tSeedLabs.append('total')
                tSeedTypes.append('-')
                simCount = sum(simVals)
                simVals.append(simCount)
                roundDF[f'round {idx}', 'key'] = tSeeds
                roundDF[f'round {idx}', 'val'] = simVals
                roundDF[f'round {idx}', 'label'] = tSeedLabs
                roundDF[f'round {idx}', 'type'] = tSeedTypes
                
                sumVal += simCount
                maxVal = max(simCount, maxVal)
                minVal = min(simCount, minVal)
        
            roundDF.to_csv(os.path.join(ICSimDir, f'{softwareName}.{methodName}.sim.csv'))
            meanVal = sumVal / ICSimCount
            df.loc[methodName, 'max'] = maxVal
            df.loc[methodName, 'min'] = minVal
            df.loc[methodName, 'mean'] = meanVal
            df.loc[methodName, 'sum'] = sumVal
        loc += 1
        
df